# FP-Growth Modeling for Association Rules

Notebook này sử dụng ma trận `basket_bool` (đã chuẩn bị ở Notebook basket_preparation.ipynb) để:

- Khai thác tập mục phổ biến bằng thuật toán FP-Growth
- Sinh luật kết hợp (association rules) từ các frequent itemsets
- So sánh kết quả với Apriori (về số lượng luật, tốc độ, chất lượng)
- Trực quan hóa các luật tiêu biểu phục vụ storytelling & phân tích kinh doanh

Notebook được thiết kế theo kiểu parameterized để dễ dàng tích hợp với **papermill**.

In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn tới basket_bool được tạo từ Notebook 02
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Đường dẫn lưu file luật kết hợp sau khi lọc (FP-Growth)
RULES_FPG_OUTPUT_PATH = "data/processed/rules_fpgrowth_filtered.csv"

# Tham số cho bước khai thác tập mục phổ biến
MIN_SUPPORT = 0.01       # ngưỡng support tối thiểu
MAX_LEN = 3              # độ dài tối đa itemset (số sản phẩm trong 1 tập)

# Tham số generate rules
METRIC = "lift"          # 'support', 'confidence' hoặc 'lift'
MIN_THRESHOLD = 1.0      # ngưỡng tối thiểu cho METRIC

# Tham số lọc luật sau khi generate
FILTER_MIN_SUPPORT = 0.01
FILTER_MIN_CONF = 0.3
FILTER_MIN_LIFT = 1.2
FILTER_MAX_ANTECEDENTS = 2
FILTER_MAX_CONSEQUENTS = 1

# Số lượng luật top để vẽ
TOP_N_RULES = 20

# Bật/tắt các biểu đồ matplotlib
PLOT_TOP_LIFT = True
PLOT_TOP_CONF = True
PLOT_SCATTER = True
PLOT_NETWORK = True

# Bật/tắt biểu đồ HTML tương tác (Plotly)
PLOT_PLOTLY_SCATTER = True


## Set up

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.express as px

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import FPGrowthRulesMiner


## Thiết lập style vẽ biểu đồ

In [ ]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

## Tải basket_bool

In [ ]:
# Đọc ma trận basket_bool từ bước 2
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)

print("=== Thông tin basket_bool ===")
print(f"- Số hoá đơn (rows): {basket_bool.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket_bool.shape[1]:,}")
print(f"- Tỷ lệ ô = 1 (có mua): {basket_bool.values.mean():.4f}")

basket_bool.head()

## Khai phá tập phổ biến bằng thuật toán FP-Growth

In [ ]:
# Khởi tạo miner cho FP-Growth
fpg_miner = FPGrowthRulesMiner(basket_bool=basket_bool)

start_time = time.time()
frequent_itemsets_fpg = fpg_miner.mine_frequent_itemsets(
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN,
    use_colnames=True,
)
elapsed = time.time() - start_time

print("=== Frequent Itemsets (FP-Growth) ===")
print(f"- Thời gian chạy:       {elapsed:.2f} giây")
print(f"- Số frequent itemsets: {frequent_itemsets_fpg.shape[0]:,}")

frequent_itemsets_fpg.head(10)


## Sinh luật kết hợp từ tập mục phổ biến

In [ ]:
rules_fpg = fpg_miner.generate_rules(
    metric=METRIC,
    min_threshold=MIN_THRESHOLD,
)

# Thêm cột dạng chuỗi dễ đọc
rules_fpg = fpg_miner.add_readable_rule_str()

print("=== Một vài luật kết hợp đầu tiên (chưa lọc) ===")
cols_preview = [
    "antecedents_str",
    "consequents_str",
    "support",
    "confidence",
    "lift",
]
rules_fpg[cols_preview].head(10)

## Lọc các luật theo ngưỡng support/confidence/lift

In [ ]:
rules_filtered_fpg = fpg_miner.filter_rules(
    min_support=FILTER_MIN_SUPPORT,
    min_confidence=FILTER_MIN_CONF,
    min_lift=FILTER_MIN_LIFT,
    max_len_antecedents=FILTER_MAX_ANTECEDENTS,
    max_len_consequents=FILTER_MAX_CONSEQUENTS,
)

print("=== Thống kê sau khi lọc luật ===")
print(f"- Tổng số luật ban đầu: {rules_fpg.shape[0]:,}")
print(f"- Số luật sau khi lọc: {rules_filtered_fpg.shape[0]:,}")

rules_filtered_fpg[cols_preview].head(10)


## Trực quan top các luật theo Lift

In [ ]:
if PLOT_TOP_LIFT and not rules_filtered_fpg.empty:
    top_rules_lift = rules_filtered_fpg.sort_values(
        "lift", ascending=False
    ).head(TOP_N_RULES)

    plt.figure(figsize=(10, min(0.4 * len(top_rules_lift), 10)))
    plt.barh(top_rules_lift["rule_str"], top_rules_lift["lift"])
    plt.xlabel("Lift")
    plt.ylabel("Luật")
    plt.title(f"Top {len(top_rules_lift)} luật theo Lift (Apriori)")
    plt.gca().invert_yaxis()  # luật lớn nhất nằm trên
    plt.tight_layout()
    plt.show()
else:
    if rules_filtered_fpg.empty:
        print("Không có luật nào sau khi lọc để vẽ top lift.")
    else:
        print("PLOT_TOP_LIFT = False, bỏ qua biểu đồ top lift.")

## Trực quan top các luật theo confidence

In [ ]:
if PLOT_TOP_CONF and not rules_filtered_fpg.empty:
    top_rules_conf = rules_filtered_fpg.sort_values(
        "confidence", ascending=False
    ).head(TOP_N_RULES)

    plt.figure(figsize=(10, min(0.4 * len(top_rules_conf), 10)))
    plt.barh(top_rules_conf["rule_str"], top_rules_conf["confidence"])
    plt.xlabel("Confidence")
    plt.ylabel("Luật")
    plt.title(f"Top {len(top_rules_conf)} luật theo Confidence (Apriori)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    if rules_filtered_fpg.empty:
        print("Không có luật nào sau khi lọc để vẽ top confidence.")
    else:
        print("PLOT_TOP_CONF = False, bỏ qua biểu đồ top confidence.")


In [ ]:
# Trực quan hoá quan hệ support vs confidence
if PLOT_SCATTER and not rules_filtered_fpg.empty:
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(
        rules_filtered_fpg["support"],
        rules_filtered_fpg["confidence"],
        c=rules_filtered_fpg["lift"],
        s=40,
        alpha=0.7,
    )
    plt.colorbar(scatter, label="Lift")
    plt.xlabel("Support")
    plt.ylabel("Confidence")
    plt.title("Phân bố luật: Support vs Confidence (màu = Lift)")
    plt.tight_layout()
    plt.show()
else:
    if rules_filtered_fpg.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter.")
    else:
        print("PLOT_SCATTER = False, bỏ qua biểu đồ scatter.")


In [ ]:
## Tạo biểu đồ bằng HTML
if PLOT_PLOTLY_SCATTER and not rules_filtered_fpg.empty:
    fig = px.scatter(
        rules_filtered_fpg,
        x="support",
        y="confidence",
        color="lift",
        size="lift",
        hover_name="rule_str",
        title="Biểu đồ tương tác: Support vs Confidence (màu & kích thước = Lift)",
        labels={
            "support": "Support",
            "confidence": "Confidence",
            "lift": "Lift",
        },
    )
    fig.show()
else:
    if rules_filtered_fpg.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter Plotly.")
    else:
        print("PLOT_PLOTLY_SCATTER = False, bỏ qua biểu đồ Plotly.")


In [ ]:
## Network các luật có lift cao
if PLOT_NETWORK and not rules_filtered_fpg.empty:
    # Lấy một tập luật nhỏ để vẽ mạng (tránh quá rối)
    top_network_rules = rules_filtered_fpg.sort_values(
        "lift", ascending=False
    ).head(min(TOP_N_RULES, 30))

    G = nx.DiGraph()

    for _, row in top_network_rules.iterrows():
        for a in row["antecedents"]:
            for c in row["consequents"]:
                # có thể gán weight theo lift
                G.add_edge(a, c, weight=row["lift"])

    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G, k=0.5, seed=42)

    edges = G.edges(data=True)
    weights = [d["weight"] for (_, _, d) in edges] if edges else [1]

    nx.draw_networkx_nodes(G, pos, node_size=1500, node_color="lightblue")
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight="bold")
    nx.draw_networkx_edges(
        G,
        pos,
        arrowstyle="->",
        arrowsize=15,
        width=[w / max(weights) * 2 for w in weights],
        edge_color="gray",
    )

    plt.title("Mạng lưới các luật kết hợp (Arrow: antecedent → consequent)")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
else:
    if rules_filtered_fpg.empty:
        print("Không có luật nào sau khi lọc để vẽ network graph.")
    else:
        print("PLOT_NETWORK = False, bỏ qua network graph.")


In [ ]:
## Lưu luật đã lọc ra file CSV

# Lưu luật đã lọc để dùng trong báo cáo / dashboard
fpg_miner.save_rules(
    output_path=RULES_FPG_OUTPUT_PATH,
    rules_df=rules_filtered_fpg,
)

print("Đã lưu luật Apriori đã lọc:")
print(f"- File: {RULES_FPG_OUTPUT_PATH}")
print(f"- Số luật: {rules_filtered_fpg.shape[0]:,}")
